In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os.path
from scipy.stats import boxcox, norm

from IPython.core.display import display

%matplotlib inline

sns.set()

tf.set_random_seed(123456789)
np.random.seed(123456789)

plt.rcParams['font.family'] = 'Yu Mincho'

# C:\\sample\\auto_encoder にデータを読み書きする
WORK_DIR = os.path.join('C:\\', 'sample', 'auto_encoder')

ソースコード4.25　データの読み込み

In [ ]:
print('データ読み込み開始')
data = pd.read_pickle(
    os.path.join(
        WORK_DIR,
        'data',
        'excess_returns_with_financial_data.pickle'
    )
)
print('データ読み込み終了')

# 欠損データの割合を確認
display(
    (
        data.isna().sum(axis=0) / data.shape[0]
    ).sort_values(ascending=False)[:20]
)

data_without_na = data.select_dtypes(include='number').dropna()

ソースコード4.26　各カラムのヒストグラムプロット

In [ ]:
fig, axes = plt.subplots(10, 4, figsize=(20, 30))

for column, ax in zip(data_without_na.columns, axes.flatten()):
    sns.distplot(
        data_without_na.loc[:, column],
        ax=ax
    ).set_title(
        f'項目「{column}」のヒストグラム'
    )

ソースコード4.27　カラム毎の標準化とボックス・コックス変換

In [ ]:
data_normed = (
    data_without_na - data_without_na.min(axis=0)
) / (
    data_without_na.max(axis=0) - data_without_na.min(axis=0)
)

data_shifted = data_normed + data_normed.min(axis=0) + 1e-08
data_bc = pd.DataFrame({
    col: boxcox(
        data_shifted.loc[:, col]
    )[0] for col in data_shifted.columns
})

ソースコード4.28　ボックス・コックス変換後のヒストグラムプロット

In [ ]:
fig, axes = plt.subplots(10, 4, figsize=(20, 30))

for column, ax in zip(data_bc.columns, axes.flatten()):
    sns.distplot(
        data_bc.loc[:, column].clip(
            data_bc.loc[:, column].mean() - data_bc.loc[:, column].std()*3,
            data_bc.loc[:, column].mean() + data_bc.loc[:, column].std()*3
        ),
        fit=norm,
        ax=ax
    ).set_title(
        f'ボックスコックス変換後の項目「{column}」のヒストグラム'
    )

fig.tight_layout()

ソースコード4.29　ボックス・コックス変換後の標準化

In [ ]:
data_bc_normed = (
    data_bc - data_bc.min(axis=0)
) / (
    data_bc.max(axis=0) - data_bc.min(axis=0)
) * 2 - 1

* ソースコード4.30　k-スパースオートエンコーダの構築（KSparseAutoEncoderクラスから抜粋）
* ソースコード4.34　 エンコーディングとデコーディングメソッド（KSparseAutoEncoderクラスか

In [ ]:
class KSparseAutoEncoder:
    
# --- ソースコード4.30 ---
    def __init__(
        self,
        sess,
        num_columns,
        k,
        hidden_layer_dimension,
        training=False
    ):
        with tf.variable_scope('simple_autoencoder'):
            with tf.variable_scope('input_layer'):
                self._inputs = tf.placeholder(
                    tf.float32, [num_columns], name='inputs'
                )
                self._inputs_reshaped = tf.expand_dims(
                    self._inputs,
                    0
                )
            
            # エンコーダ
            with tf.variable_scope('encoder'):
                self._weights = tf.get_variable(
                    'weights',
                    [num_columns, hidden_layer_dimension],
                    dtype=tf.float32
                )
                self._encoder_bias = tf.get_variable(
                    'encoder_bias',
                    [hidden_layer_dimension],
                    dtype=tf.float32
                )
                
                self._hidden_layer = \
                self._inputs_reshaped @ self._weights + self._encoder_bias
                
                self._top_k_values, self.top_k_indices = tf.nn.top_k(
                    self._hidden_layer, k=k
                )
                
                self._encoding = tf.scatter_nd(
                    indices=tf.expand_dims(
                        tf.squeeze(self.top_k_indices), axis=-1
                    ),
                    updates=tf.squeeze(self._top_k_values),
                    shape=tf.shape(tf.squeeze(self._hidden_layer))
                )
                
            with tf.variable_scope('decoder'):
                self._encoding_reshaped = tf.expand_dims(self._encoding, 0)
                self._decoder_bias = tf.get_variable(
                    'decoder_bias',
                    [num_columns],
                    dtype=tf.float32
                )
                
                # ウェイトを使い回す
                self._outputs = \
                self._encoding_reshaped @ tf.transpose(self._weights) + self._decoder_bias
            
            self._loss = tf.losses.mean_squared_error(
                self._inputs_reshaped,
                self._outputs
            )
            self._training = tf.train.AdamOptimizer().minimize(self._loss)
            
        self.saver = tf.train.Saver()
        if training:
            sess.run(tf.global_variables_initializer())
# --- ソースコード4.30 ここまで ---
            
    def train_one_step(self, sess, inputs):
        feed_dict = {
            self._inputs: inputs
        }
        fetches = [
            self._loss,
            self._training
        ]
        return sess.run(fetches, feed_dict)

# --- ソースコード4.34 ---
    def encode(self, sess, inputs):
        feed_dict = {self._inputs: inputs}
        fetches = [self._encoding]
        return sess.run(fetches, feed_dict)[0]
    
    def decode(self, sess, encoding):
        feed_dict = {self._encoding: encoding}
        fetches = [self._outputs]
        return np.squeeze(sess.run(fetches, feed_dict)[0])
# --- ソースコード4.34 ここまで ---
    
    def get_weights(self, sess):
        fetches = [self._weights]
        return sess.run(fetches)[0]
    
    def get_encoder_bias(self, sess):
        fetches = [self._encoder_bias]
        return sess.run(fetches)
    
    def get_decoder_bias(self, sess, inputs):
        feed_dict = {self._inputs: inputs}
        fetches = [self._decoder_bias]
        return sess.run(fetches, feed_dict)
    
    def save(self, sess, path):
        self.saver.save(sess, path)
        print(f'model saved in {path}')
        
    def restore(self, sess, path):
        self.saver.restore(sess, path)
        print(f'model restored from {path}')

ソースコード4.31　学習用メタパラメータの定義

In [ ]:
encoding_dim = 20
num_columns = data_bc_normed.shape[1]
max_training_steps = data_bc_normed.shape[0]
steps_per_epoch = 10000

ソースコード4.32　オートエンコーダの学習

In [ ]:
loss_over_time = np.empty(max_training_steps // steps_per_epoch)
tf.reset_default_graph()

with tf.Graph().as_default() and tf.Session() as session:
    auto_encoder = KSparseAutoEncoder(
        session,
        num_columns,
        encoding_dim,
        hidden_layer_dimension=num_columns * 4,
        training=True
    )
    training_step = 0    
    loss_mean_epoch = 0.
    for random_row in np.random.choice(
        data_bc_normed.shape[0],
        max_training_steps,
        replace=True  # 同じデータが数回選択されることを許す
    ):
        current_loss, _ = auto_encoder.train_one_step(
            session,
            data_bc_normed.iloc[random_row, :].values  # 期待出力は不要
        )
        loss_mean_epoch += current_loss
        training_step += 1
        if training_step % steps_per_epoch == 0:
            loss_over_time[
                training_step // steps_per_epoch - 1
            ] = loss_mean_epoch / steps_per_epoch
            
            print(
                f'[{(training_step * 100) // max_training_steps:3}%] '
                f'直近{steps_per_epoch}ステップの平均ロス：'
                f'{loss_over_time[training_step // steps_per_epoch - 1]:.5f}'
            )
            loss_mean_epoch = 0.
            
    auto_encoder.save(
        session,
        os.path.join(WORK_DIR, 'models', 'simple_auto_encoder', 'simple_auto_encoder'))
loss_over_time_bkp = loss_over_time.copy()

ソースコード4.33　ロスの時系列プロット

In [ ]:
loss_df = pd.DataFrame(
    loss_over_time_bkp, columns=['ロス']
).rename_axis('学習エポック')
loss_df = loss_df.assign(ロス移動平均=loss_df.rolling(10).mean())

fig, axes = plt.subplots(2, 1, figsize=(12, 10))
grid = sns.lineplot(data=loss_df, ax=axes[0]).set_title(
    'オートエンコーダ学習ロス時系列'
)
# plt.show()

# _, ax = plt.subplots(figsize=(12, 5))
sns.lineplot(data=loss_df[20:], ax=axes[1]).set_title(
    'オートエンコーダ学習ロス時系列（20エポック目以降）'
)
fig.tight_layout()

ソースコード4.35　入力とオートエンコーダの出力を比較するデータフレーム作成

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default() and tf.Session() as session:
    auto_encoder = KSparseAutoEncoder(
        session,
        num_columns,
        encoding_dim,
        hidden_layer_dimension=num_columns * 4,
        training=False
    )
    auto_encoder.restore(
        session,
        os.path.join(
            WORK_DIR, 'models', 'simple_auto_encoder', 'simple_auto_encoder')
    )
    
    row_i = np.random.randint(data_bc_normed.shape[0])
    input_values = data_bc_normed.iloc[row_i, :].values
    encoding = auto_encoder.encode(session, input_values)
    decoded_encoding = auto_encoder.decode(session, encoding)

inputs_and_reconstructed = pd.DataFrame(
    data=np.transpose([input_values, decoded_encoding]),
    columns=['入力', 'NN出力']
).set_index(
    data_bc_normed.columns
).stack().reset_index()

inputs_and_reconstructed.columns = ['カラム', '入力またはNN出力', '値']
display(inputs_and_reconstructed.head())

ソースコード4.36　入力値とオートエンコーダによって再現された値のプロット作成

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

sns.barplot(
    ax=ax,
    data=inputs_and_reconstructed,
    x='カラム',
    y='値',
    hue='入力またはNN出力'
).set_title(
    '入力とそれに対してのオートエンコーダ出力（正規化、標準化後）'
)

plt.xticks(rotation=90)
plt.show()

ソースコード4.37　オートエンコーダのウェイト行列のヒートマップ作成

In [ ]:
tf.reset_default_graph()
with tf.Graph().as_default() and tf.Session() as session:
    auto_encoder = KSparseAutoEncoder(
        session,
        num_columns,
        encoding_dim,
        hidden_layer_dimension=num_columns * 4,
        training=False
    )
    auto_encoder.restore(
        session,                
        os.path.join(
            WORK_DIR, 'models', 'simple_auto_encoder', 'simple_auto_encoder'
        )
    )
    
    weights = auto_encoder.get_weights(session)
    
    weights_df = pd.DataFrame(
        weights,
        columns=[f'変数{v}' for v in range(weights.shape[1])],
        index=data_bc_normed.columns
    )

fig, ax = plt.subplots(figsize=(20, 12))
sns.heatmap(weights_df, ax=ax, cmap='PiYG', center=0.)
plt.show()